In [1]:
import os

In [2]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [3]:
os.chdir('../')

In [4]:
import pandas as pd

In [11]:
from dataclasses import dataclass
from pathlib import Path    


# Fix DataSpilter to DataSplitter in the config class
@dataclass
class DataSplitterConfig:  # Changed from DataSpilterConfig
    root_dir: Path
    train_data: Path
    test_data: Path
    split_ratio: float
    random_state: int
    unzip_data_dir: Path  # Add this missing field

In [12]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [13]:
class DataSplitter:  
    def __init__(self, config: DataSplitterConfig):
        self.config = config

    def split_data(self):  # Fixed method name from spilt_data
        df = pd.read_csv(self.config.unzip_data_dir)

        train_df = df.sample(
            frac=self.config.split_ratio,  # Changed from 1 - split_ratio
            random_state=self.config.random_state
        )
        test_df = df.drop(train_df.index)

        train_df.to_csv(self.config.train_data, index=False)
        test_df.to_csv(self.config.test_data, index=False)

        logger.info(f"Train and Test data saved in {self.config.root_dir}")

        return self.config.train_data, self.config.test_data

In [14]:
from src.IPL_Predictor import logger

In [15]:
class DataSplitter:  # Changed from DataSpilter
    def __init__(self, config: DataSplitterConfig):
        self.config = config

    def split_data(self):  # Fixed method name from spilt_data
        df = pd.read_csv(self.config.unzip_data_dir)

        train_df = df.sample(
            frac=self.config.split_ratio,  # Changed from 1 - split_ratio
            random_state=self.config.random_state
        )
        test_df = df.drop(train_df.index)

        train_df.to_csv(self.config.train_data, index=False)
        test_df.to_csv(self.config.test_data, index=False)

        logger.info(f"Train and Test data saved in {self.config.root_dir}")

        return self.config.train_data, self.config.test_data

In [16]:
try:
    config = ConfigurationManager()
    data_splitting_config = config.get_data_splitter_config()
    data_validation = DataSplitter(config=data_splitting_config)
    data_validation.split_data()
except Exception as e:
    logger.error(f"Error occurred during data validation: {e}")

[2025-09-21 18:30:23,505: ERROR: 1510746215: Error occurred during data validation: name 'ConfigurationManager' is not defined]
